In [25]:
import folium
import pandas as pd
import geopandas as gpd
from folium import plugins
from shapely.geometry import Point,shape
from branca.colormap import linear

# Trash Collecting Data

In [26]:
trash = pd.read_csv('trash.csv')

trash.head()

,Request_ID,Create_Date,Request_Type_ID,Request_Type,Department,Street_ID,Longitude,Latitude,Ward_Precinct
0,59225,2/13/19 16:37,341,Trash - Bag Request,Highway,647,-71.084017,42.751749,7_3
1,52598,11/20/18 17:53,341,Trash - Bag Request,Highway,305,-71.086570,42.767473,2_1
2,86005,1/21/20 15:10,341,Trash - Bag Request,Highway,630,-71.055604,42.748844,7_3
3,25958,1/19/18 16:42,341,Trash - Bag Request,Highway,559,-71.077883,42.789527,3_3
4,61378,3/7/19 16:35,342,Trash - Barrel Overflow,Highway,1061,-71.081813,42.777646,3_1


In [27]:
ward_df = gpd.read_file('Wards/Hav_Precincts_Wards_WGS84.json',driver = 'GeoJSON')
ward_df = ward_df.to_crs(epsg ='4326')
ward_df = ward_df[['Precinct','geometry']]
data_df = trash[['Request_ID','Ward_Precinct','Longitude','Latitude']]

In [28]:
count_df = data_df.groupby('Ward_Precinct')['Request_ID'].nunique().reset_index()
count_df.rename(columns={'Ward_Precinct':'Precinct'}, inplace=True)
count_df.rename(columns={'Request_ID':'nb'}, inplace=True)

"""Using the 2011 map data"""
ward_df = ward_df.tail(21)
nbh_geo_count_df = ward_df.sort_values(by = 'Precinct').reset_index()
nbh_geo_count_df['nb'] = count_df['nb']
nbh_geo_count_df = nbh_geo_count_df[['Precinct','geometry','nb']]
nbh_geo_count_df.head()

,Precinct,geometry,nb
0,1-1,"POLYGON ((-71.09160 42.77516, -71.09232 42.774...",45
1,1-2,"POLYGON ((-71.08679 42.78449, -71.08710 42.785...",46
2,1-3,"POLYGON ((-71.10464 42.79081, -71.10478 42.790...",24
3,2-1,"POLYGON ((-71.06884 42.77243, -71.06980 42.773...",27
4,2-2,"POLYGON ((-71.12075 42.76855, -71.12138 42.768...",10


In [39]:
trash_map = folium.Map(
    location = [42.7762,-71.0773],
    zoom_start = 13)

nbh_count_colormap = linear.OrRd_03.scale(min(count_df['nb']),
                                            max(count_df['nb']))

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.2
}

folium.GeoJson(
    nbh_geo_count_df,name = 'number of requests',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['Precinct', 'nb'],
        aliases=['Precinct id', 'Number of Requests'],
        localize=True
    )
).add_to(trash_map)

df = gpd.read_file('Refuse_route/Hav_Refuse_Routes_WGS84.shx')
df = df.to_crs(epsg ='4326')
js = df.to_json()

style_function = lambda x: {'color': 'green',
                           'weight': 2.5,
                           'fillOpacity':0}
folium.GeoJson(js,name = 'Refuse_routes',
               style_function = style_function).add_to(trash_map)

add clusers of data points

In [40]:
locs_geometry = [Point(xy) for xy in zip(trash.Longitude,trash.Latitude)]
crs = {'init':'epsg:4326'}
locs_gdf = gpd.GeoDataFrame(trash,crs = crs,geometry= locs_geometry)

/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [41]:
feature_id = folium.FeatureGroup(name = 'Illegal Dumping - Black')
feature_bo = folium.FeatureGroup(name = 'Barrel Overflow - Purple')
feature_e = folium.FeatureGroup(name = 'Enforcement - Red')
feature_br = folium.FeatureGroup(name = 'Bag Request - Blue')
feature_tcm = folium.FeatureGroup(name = 'Trash Cart Management - Yellow')

marker_id = plugins.MarkerCluster().add_to(feature_id)
marker_bo = plugins.MarkerCluster().add_to(feature_bo)
marker_e = plugins.MarkerCluster().add_to(feature_e)
marker_br = plugins.MarkerCluster().add_to(feature_br)
marker_tcm = plugins.MarkerCluster().add_to(feature_tcm)

for i, v in locs_gdf.iterrows():
    
    if v['Request_Type'] == 'Trash - Illegal Dumping':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=3,
                            color='black',
                            fill_color='black',
                            fill=True).add_to(marker_id)
    if v['Request_Type'] == 'Trash - Barrel Overflow':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=3,
                            color='purple',
                            fill_color='purple',
                            fill=True).add_to(marker_bo)
    if v['Request_Type'] == 'Trash - Enforcement':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=3,
                            color='red',
                            fill_color='red',
                            fill=True).add_to(marker_e)
    if v['Request_Type'] == 'Trash - Bag Request':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=3,
                            color='blue',
                            fill_color='blue',
                            fill=True).add_to(marker_br)
    if v['Request_Type'] == 'Trash Cart Management':
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=3,
                            color='yellow',
                            fill_color='yellow',
                            fill=True).add_to(marker_tcm)
                                              
feature_id.add_to(trash_map)
feature_bo.add_to(trash_map)
feature_e.add_to(trash_map)
feature_br.add_to(trash_map)
feature_tcm.add_to(trash_map)
folium.LayerControl(collapsed=False).add_to(trash_map)
trash_map


In [42]:
trash_map.save("trash_issues.html")